In [1]:
import tensorflow as tf

In [2]:
data_URL = 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt'

data_path = tf.keras.utils.get_file("dataset.txt", data_URL)

text = open(data_path, "rb").read().decode("utf-8").lower()

text = text[300000:800000]

1115394/1115394 ━━━━━━━━━━━━━━━━━━━━ 1s 1us/step


In [7]:
vocab = sorted(set(text))

char_to_index = {c: i for i, c in enumerate(vocab)}
index_to_char = {i:c for i, c in enumerate(vocab)}

In [14]:
sentences = []
next_char = []

sequenceSize = 40
stepSize = 3

for i in range(0, len(text)-sequenceSize, stepSize):
    sentences.append(text[ i : i+sequenceSize ])
    next_char.append(text[ i : i+sequenceSize ])